In [1]:
%use krangl(0.15.2)

In [2]:
val userHome = System.getProperty("user.home")

println(userHome)

if(userHome.contains("brandl")){
    println("clearing ivy cache")
    java.io.File("$userHome/.ivy2/cache/org.kalasim/").deleteRecursively()
}


C:\Users\brandl
clearing ivy cache


In [3]:
@file:DependsOn("org.kalasim:kalasim:0.6")

In [10]:
// @file:Repository("*mavenLocal")
// @file:DependsOn("org.kalasim:kalasim:0.6-SNAPSHOT")
// :classpath

In [11]:
import jetbrains.letsPlot.geom.geom_boxplot
import jetbrains.letsPlot.geom.geom_density
import jetbrains.letsPlot.geom.geom_point
import jetbrains.letsPlot.ggsize
import jetbrains.letsPlot.intern.Plot
import jetbrains.letsPlot.lets_plot
import krangl.irisData
import krangl.toMap
import org.kalasim.demo.MM1Queue
import org.kalasim.plot.letsplot.display
import org.kalasim.InteractionEvent



In [5]:
val mm1 = MM1Queue().apply {
    run(100)
}

rho is 0.75. With rho>1 the system would be unstable, because there are more arrivals then the server can serve.


In [12]:
 mm1.traces.filterIsInstance<InteractionEvent>().asDataFrame()

action,curComponent,details,source,logLevel,time
Created,main,capacity=1,server,INFO,0.0
create,main,,ComponentGenerator.1,INFO,0.0
activate,main,scheduled for .00,ComponentGenerator.1,INFO,0.0
run +100.00,main,scheduled for 100.00,main,INFO,0.0
hold +.68,ComponentGenerator.1,scheduled for .68,ComponentGenerator.1,INFO,0.0
create,ComponentGenerator.1,,Customer.1,INFO,0.6826910400986043
activate,ComponentGenerator.1,scheduled for .68,Customer.1,INFO,0.6826910400986043
hold +.55,ComponentGenerator.1,scheduled for 1.23,ComponentGenerator.1,INFO,0.6826910400986043
Requesting 1.0 from server with priority null a...,Customer.1,null,Customer.1,INFO,0.6826910400986043
null,Customer.1,null,Customer.1,INFO,0.6826910400986043


In [7]:
mm1.server.claimedMonitor.display()


In [8]:
mm1.server.requesters.lengthOfStayMonitor.display()